In [1]:
import numpy as np
from IPython import display
import gpiozero
import time

In [2]:
# Defining GPIO inputs and outputs
pwm_l = gpiozero.PWMOutputDevice(pin=18,active_high=True,initial_value=0,frequency=50000)
dir_l = gpiozero.OutputDevice(pin=23)

pwm_r = gpiozero.PWMOutputDevice(pin=15,active_high=True,initial_value=0,frequency=50000)
dir_r = gpiozero.OutputDevice(pin=14)

encoder_l = gpiozero.RotaryEncoder(a=5, b=6,max_steps=100000) 
encoder_r = gpiozero.RotaryEncoder(a=25, b=8,max_steps=100000) 


# add sensors

In [3]:
# Defining robot model class
class DiffDriveRobot:
    
    def __init__(self, dt=0.1, wheel_radius=0.028, wheel_sep=0.101):
        
        self.x = 0.0 # y-position
        self.y = 0.0 # y-position 
        self.th = 0.0 # orientation
        
        self.wl = 0.0 # rotational velocity left wheel
        self.wr = 0.0 # rotational velocity right wheel
        
        self.steps_l = 0 # total right encoder steps
        self.steps_r = 0 # total left encoder steps
        
        self.dt = dt
        self.r = wheel_radius
        self.l = wheel_sep

    
    # Retrieving step count from encoders. 
    # If encoder steps higher, then wheel has gone clockwise.
    # This is forward movement for right wheel and backwards for left wheel, 
    # so multiply by -1 for left.
    def get_steps(self, encoder, side):
        if (side == 0):     # zero for left side
            steps_curr = encoder.steps - self.steps_l
            self.steps_l = encoder.steps
            return (steps_curr)
        elif (side == 1):
            steps_curr = encoder.steps - self.steps_r
            self.steps_r = encoder.steps
            return steps_curr
        
    # Calculate motor angular velocity from shaft encoder feedback
    def wheel_speed(self, steps):

        N = 128 # Encoder state transitions
        RATIO = 114.7 # Gear ratio
        
        steps_ps = steps/self.dt
        rps_encoder = steps_ps/N
        rps_wheel = rps_encoder/RATIO
        w = 2*np.pi*rps_wheel
        
        return w
    
    # Velocity motion model
    # Calculate base linear and angular velocity from left and right wheel angular velocities
    def base_velocity(self, wl, wr):
        
        v = (wl*self.r + wr*self.r)/2.0
        w = (wl - wr)/self.l
        
        return v, w
    
    # Kinematic motion model
    def pose_update(self, duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r):
        
        # Get current steps
        steps_curr_l = self.get_steps(encoder_l, 0)
        steps_curr_r = self.get_steps(encoder_r, 1)
        
        # Update angular velocities
        self.wl = self.wheel_speed(steps_curr_l)
        self.wr = self.wheel_speed(steps_curr_r)
        
        # Update motor output
        pwm_l.value = duty_cycle_l
        pwm_r.value = duty_cycle_r
        
#         pwm_l.value = abs(duty_cycle_l)
#         pwm_r.value = abs(duty_cycle_r)
        
#         if duty_cycle_l >=0:
#             dir_l.value = 0
#         else: dir_l.value = 1
#         if duty_cycle_r >=0:
#             dir_r.value = 0
#         else: dir_r.value = 1
        
        # Update robot position
        v, w = self.base_velocity(self.wl,self.wr)
        self.x = self.x + self.dt*v*np.cos(self.th)
        self.y = self.y + self.dt*v*np.sin(self.th)
        self.th = self.th + w*self.dt
        
        
        return self.x, self.y, self.th
        

In [4]:
# Defining robot velocity controller class
class RobotController:
    
    def __init__(self, Kp=0.5, Ki=0.125, wheel_radius=0.028, wheel_sep=0.101):
        
        self.Kp = Kp
        self.Ki = Ki
        self.r = wheel_radius
        self.l = wheel_sep
        self.e_sum_l = 0 # error term of left wheel
        self.e_sum_r = 0 # error term of right wheel
        
    # Implement duty cycle PI control given target and actual angular velocity
    def p_control(self,w_desired,w_measured,e_sum):
        
        duty_cycle = min(max(-1,self.Kp*(w_desired-w_measured) + self.Ki*e_sum),1)
        e_sum = e_sum + (w_desired-w_measured)
        
        
        return duty_cycle, e_sum
        
        
    # Calculate duty cycles from target linear and angular velocity 
    def drive(self,v_desired,w_desired,wl,wr):
        
        wl_desired = v_desired/self.r + self.l*w_desired/2 
        wr_desired = v_desired/self.r - self.l*w_desired/2
        
        duty_cycle_l,self.e_sum_l = self.p_control(wl_desired,wl,self.e_sum_l)
        duty_cycle_r,self.e_sum_r = self.p_control(wr_desired,wr,self.e_sum_r)
        
        
        return duty_cycle_l, duty_cycle_r
        

In [5]:
# Defining robot path planner class (without obstacle avoidance)
class TentaclePlanner:
    
    def __init__(self,dt=0.1,steps=5,alpha=1,beta=0.1):
        
        self.dt = dt
        self.steps = steps
        # Tentacles are possible trajectories to follow
        self.tentacles = [(0.0,0.5),(0.0,-0.5),(0.1,1.0),(0.1,-1.0),(0.1,0.5),(0.1,-0.5),(0.1,0.0),(0.0,0.0)]
        
        self.alpha = alpha
        self.beta = beta
    
    # Play a trajectory and evaluate where you'd end up
    def roll_out(self,v,w,goal_x,goal_y,goal_th,x,y,th):
        
        for j in range(self.steps):
        
            x = x + self.dt*v*np.cos(th)
            y = y + self.dt*v*np.sin(th)
            th = (th + w*self.dt)
        
        # Wrap angle error -pi,pi
        e_th = goal_th-th
        e_th = np.arctan2(np.sin(e_th),np.cos(e_th))
        
        cost = self.alpha*((goal_x-x)**2 + (goal_y-y)**2) + self.beta*(e_th**2)
        
        return cost
    # Choose trajectory that will get you closest to the goal
    def plan(self,goal_x,goal_y,goal_th,x,y,th):
        
        costs = []
        for v,w in self.tentacles:
            costs.append(self.roll_out(v,w,goal_x,goal_y,goal_th,x,y,th))
        
        best_idx = np.argmin(costs)
        
        return self.tentacles[best_idx]
        

In [6]:
# Initialising models
dt = 0.1
r = 0.028
sep = 0.101


robot = DiffDriveRobot(dt = dt, wheel_radius = r, wheel_sep = sep)
controller = RobotController(Kp = 0.5, Ki = 0.125, wheel_radius = r, wheel_sep = sep)
planner = TentaclePlanner(dt = dt, steps = 5, alpha = 1, beta = 1e-5)

In [7]:
# # Example controller

# for i in range(300):
    
# #     if i < 100: # drive in circular path (turn left) for 10 s
# #         duty_cycle_l,duty_cycle_r = controller.drive(0.1,1,robot.wl,robot.wr)
#     if i < 200: # drive in circular path (turn right) for 10 s
#         duty_cycle_l,duty_cycle_r = controller.drive(0.1,-1,robot.wl,robot.wr)
#     else: # stop
#         duty_cycle_l,duty_cycle_r = (0,0)
        
#     # Drive robot motion - send duty cycle command to robot
#     x,y,th  = robot.pose_update(duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r)
#     time.sleep(dt)
    
            
   
#     print('Duty cycle left:',duty_cycle_l,'Duty cycle right:',duty_cycle_r)
#     print('Direction left:',dir_l.value,'Direction right:',dir_r.value)
#     print('Velocity left:',robot.wl,'Velocity right:',robot.wr)
#     print('Steps left:',robot.steps_l,'Steps right:',robot.steps_r,'\n')
    

In [8]:
# Navigation using path planning
poses = []
velocities = []
duty_cycle_commands = []

goal_x = 0.3
goal_y = 0.3
goal_th = 0

for i in range(200):

    # Plan using tentacles
    v, w = planner.plan(goal_x, goal_y, goal_th, robot.x, robot.y, robot.th)
    
    # Calculate duty cycle from required velocities
    duty_cycle_l,duty_cycle_r = controller.drive(v, w, robot.wl, robot.wr)
    
    print('Duty cycle left:',duty_cycle_l,'Duty cycle right:',duty_cycle_r)
    print('Velocity left:',robot.wl,'Velocity right:',robot.wr)
    print('Velocity goal:',v, 'Angular velocity goal:',w)
    print('Steps left:',robot.steps_l,'Steps right:',robot.steps_r)
    
    # Output duty cycles to robot
    x, y, th = robot.pose_update(duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r)
    time.sleep(dt)
    
    print('x:',x,'y:',y,'th:',th,'\n')
    
    # Log data
    poses.append([x,y,th])
    duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
    velocities.append([robot.wl,robot.wr])

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.0
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 0 Steps right: 0
x: 0.0 y: 0.0 th: 0.0 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.0
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 0 Steps right: 0
x: 0.0009526462029875326 y: 0.0 th: -0.2923709357731745 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.19258346421580846 Velocity right: 0.4878781093467148
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 45 Steps right: 114
x: 0.0016066899818164787 y: -0.00019686501098827532 th: -0.40253969418045776 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.1883038316776794 Velocity right: 0.29957427766903544
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 89 Steps right: 184
x: 0.0023233254140817513 y: -0.0005020012963209352 th: -0.5550810519751576 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.20114272929206664 Velocity

x: 0.004043134474554161 y: -0.0030115074692220966 th: -8.24570784079127 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.08987228330071062 Velocity right: 0.25249831974961556
Velocity goal: 0.0 Angular velocity goal: 0.5
Steps left: 422 Steps right: 2368
x: 0.0038441258103742875 y: -0.0034932820535917217 th: -8.402486458524713 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.10699081345322693 Velocity right: 0.2653372173640028
Velocity goal: 0.0 Angular velocity goal: 0.5
Steps left: 447 Steps right: 2430
x: 0.003591079194548714 y: -0.003907399822412315 th: -8.533841516625705 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.10699081345322693 Velocity right: 0.23965942213522834
Velocity goal: 0.0 Angular velocity goal: 0.5
Steps left: 472 Steps right: 2486
x: 0.0033085728319296897 y: -0.004256850588217525 th: -8.639773015094246 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.10699081345322693 Velocity right: 0.21398162690645386
Velocity goal: 0.0 Ang

x: -0.00020263715319593533 y: -0.004040264001221468 th: -10.533828207711773 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.004279632538129077 Velocity right: 0.11982971106761417
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 618 Steps right: 3104
x: -0.00028271551610287084 y: -0.0038793430112391857 th: -10.65247148599654 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.004279632538129077 Velocity right: 0.12410934360574324
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 619 Steps right: 3133
x: -0.0003411679052617123 y: -0.003715717061661161 th: -10.741453944710114 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.01711853015251631 Velocity right: 0.10699081345322693
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 623 Steps right: 3158
x: -0.0004059355421495171 y: -0.0034663571436408082 th: -10.872809002811106 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.025677795228774464 Velocity right: 0.15834640391077584
Ve

x: 0.0036060794453130025 y: -0.0024015368583321295 th: -13.440588525688545 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.029957427766903542 Velocity right: 0.07275375314819431
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 795 Steps right: 3967
x: 0.003679117703498733 y: -0.002488855552948025 th: -13.470249345259736 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.025677795228774464 Velocity right: 0.055635222995677996
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 801 Steps right: 3980
x: 0.003794007893412803 y: -0.0026347943254917197 th: -13.559231803973312 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.021398162690645386 Velocity right: 0.11127044599135599
Velocity goal: 0.0 Angular velocity goal: 0.5
Steps left: 806 Steps right: 4006
x: 0.0038529240917523065 y: -0.0027251259417365543 th: -13.601604403360728 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.01711853015251631 Velocity right: 0.059914855533807085
Velocity g

x: 0.0021842015242756526 y: -0.001591699463699723 th: -18.618520170830852 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0385166928431617 Velocity right: 0.12410934360574324
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1142 Steps right: 5536
x: 0.002779095130164149 y: -0.0014517589753206507 th: -18.804959608135484 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.12410934360574324 Velocity right: 0.31241317528342266
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1171 Steps right: 5609
x: 0.0031501978980423948 y: -0.0014351981796480381 th: -18.889704806910316 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.08987228330071062 Velocity right: 0.17546493406329217
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1192 Steps right: 5650
x: 0.0035932105356801477 y: -0.0014529942062832658 th: -18.957500965930183 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.12410934360574324 Velocity right: 0.19258346421580846
Velocity goa

x: 0.006406627447200619 y: -0.003066173773214418 th: -25.360000733368825 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.029957427766903542 Velocity right: 0.0770333856863234
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1567 Steps right: 7552
x: 0.006564238044694191 y: -0.0031026219285605096 th: -25.389661552940016 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.04279632538129077 Velocity right: 0.07275375314819431
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1577 Steps right: 7569
x: 0.007039413918961478 y: -0.0032274632771084563 th: -25.389661552940016 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.17546493406329217 Velocity right: 0.17546493406329217
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 1618 Steps right: 7610
x: 0.007630486347928103 y: -0.003382753735058341 th: -25.457457711959883 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.18402419913955034 Velocity right: 0.25249831974961556
Velocity goal: 

x: 0.008210424889145477 y: -0.008356244396851236 th: -28.279472831161833 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.004279632538129077 Velocity right: 0.0
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 1939 Steps right: 8613
x: 0.007874906128528045 y: -0.008354520167922673 th: -28.47438678834395 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.021398162690645386 Velocity right: 0.21826125944458294
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 1944 Steps right: 8664
x: 0.007692874384001453 y: -0.008317610480106916 th: -28.580318286812492 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.012838897614387232 Velocity right: 0.11982971106761417
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 1947 Steps right: 8692
x: 0.0075271919805789496 y: -0.008265270491858639 th: -28.6523517057711 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.025677795228774464 Velocity right: 0.09843154837696877
Velocity goal: 0.0 Angular v

In [9]:
pwm_l.off()
pwm_r.off()

In [10]:
# Defining robot path planner class (with obstacle avoidance)

In [11]:
# Run robot using predefined coordinates